In this notebook I will make 2nd solving algorithm (see readme) - it isn't done yet

In [1]:
import numpy as np
import threading
import sys

In [2]:
#sudoku = np.zeros(81,dtype="int").reshape(9,9)
def clear_sudoku():
    sudoku = [[0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0]]

    sudoku = np.array(sudoku)
    return sudoku

In [3]:
def example_easy_sudoku():
    sudoku = [[0, 7, 0, 4, 0, 5, 6, 0, 0],
              [9, 2, 4, 0, 6, 0, 8, 0, 0],
              [1, 0, 5, 0, 0, 8, 7, 0, 0],
              [0, 3, 8, 0, 5, 7, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 7, 3],
              [0, 4, 7, 0, 9, 0, 1, 2, 0],
              [4, 0, 9, 6, 0, 2, 0, 0, 1],
              [7, 0, 0, 8, 3, 0, 4, 0, 9],
              [0, 1, 0, 5, 4, 9, 0, 0, 0]]

    sudoku = np.array(sudoku)
    return sudoku

In [4]:
def example_hard_sudoku():
    sudoku = [[8, 3, 2, 1, 0, 7, 0, 9, 6],
              [0, 0, 0, 0, 0, 0, 0, 5, 0],
              [7, 4, 5, 0, 9, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 7],
              [0, 0, 0, 8, 0, 1, 0, 0, 0],
              [0, 0, 4, 0, 0, 0, 5, 0, 1],
              [3, 0, 0, 0, 0, 5, 0, 6, 2],
              [0, 6, 0, 0, 0, 8, 0, 0, 4],
              [0, 0, 0, 7, 0, 3, 0, 8, 0]]

    sudoku = np.array(sudoku)
    return sudoku

In [5]:
def example_very_hard_sudoku():
    sudoku = [[3, 7, 0, 0, 0, 0, 0, 9, 0],
              [9, 0, 0, 0, 7, 0, 0, 0, 0],
              [0, 0, 0, 4, 2, 0, 0, 0, 6],
              [0, 0, 1, 0, 8, 4, 2, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [8, 0, 0, 6, 0, 0, 0, 5, 0],
              [0, 0, 6, 0, 0, 2, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 3, 9],
              [0, 2, 0, 0, 0, 0, 4, 0, 0]]

    sudoku = np.array(sudoku)
    return sudoku

In [6]:
#sudoku[1,3] - value from row 1 and col 3
#sudoku[:,3] - whole column 3
#sudoku[1,:] - whole row 1

In [7]:
def if_possible(sudoku, row, col, number):
    possible = False
    
    ##check if number in row
    if number not in sudoku[row,:]:
        possible = True
    else:
        return False
    
    ##check if number in column
    if number not in sudoku[:,col]:
        possible = True
    else:
        return False
    
    ##check if number in squere 3x3
    start_sq_row = row // 3 #floor qoutient
    # can make it prettier
    if start_sq_row == 1:
        start_sq_row = 3
    elif start_sq_row ==2:
        start_sq_row = 6
    start_sq_col = col // 3
    if start_sq_col == 1:
        start_sq_col = 3
    elif start_sq_col ==2:
        start_sq_col = 6
        
    end_sq_row = start_sq_row + 3
    end_sq_col = start_sq_col + 3
    
#     print("Starting row of squere: {}".format(start_sq_row))
#     print("Starting column of squere: {}".format(start_sq_col))
#     print("Ending row of squere: {}".format(end_sq_row))
#     print("Ending column of squere: {}".format(end_sq_col))
    
    if number not in sudoku[start_sq_row:end_sq_row,start_sq_col:end_sq_col]:
        possible = True
    else:
        return False
    
    return possible

In [8]:
## checking if function works
#if_possible(sudoku, 1, 6, 1)

In [40]:
def restart_higher_order_put(sudoku, safe_cells, order, order_put):
    for i in range(9):
        for j in range(9):
            if order[i][j] > order_put:
                order[i][j] = 0
                sudoku[i][j] = 0
                safe_cells[i][j] = 0
    return sudoku, safe_cells, order

In [10]:
###
# pierwsze wywolanie: 
# sudoku = sudoku
# safe_cells = on_start
# rec = 1
#
# sprawdzanie kazdej komórki 
#
# sudoku jest aktualizowane co krok o te pozycje które są pewne
# safe_cells są rozszerzane o te pozycje które są pewne
# rec powiększane o 1

In [11]:
# #first try - without go back
# def solve(sudoku, safe_cells, rec):
#     if rec % 10 == 0:
#         print("Recursion: {}".format(rec))
        
#     for i in range(9):
#         for j in range(9):
#             if safe_cells[i][j] == 0:
#                 count = 0
#                 num = None
            
#                 for n in range(1,10):
#                     if if_possible(sudoku, i, j, n) == True:
#                         count = count + 1
#                         # if there will be only one possible it will be placed in next if statement
#                         num = n  

#                 if count == 1:
#                     sudoku[i][j] = num
#                     safe_cells[i][j] = 1
#     if safe_cells.sum() == 81:
#         return sudoku
#     else:
#         return solve(sudoku, safe_cells, rec+1)

In [132]:
#second try - with go back
def solve(sudoku, safe_cells, order, order_put, go_back, rec):
    print("Rec: {}".format(rec))
    print(sudoku)
    print("Safe_cells: {}".format(safe_cells))
    print("Order: {}".format(order))
    print("Order_put: {}".format(order_put))
    print("Go_back: {}".format(go_back))
    #printing recursion step
    if rec % 10 == 0:
        pass
        #print("Recursion: {}".format(rec))
        
    #if going back 
    if go_back == True:
        #print(order)
        #print("Order_put: {}".format(order_put))
        row_col = np.where(order == order_put)
        # if there is more than 1 order_put in order that means these cell had only 1 possibility so just go back a stap more
        if len(row_col[0]) > 1:
            go_back = True
            order_put = order_put - 1

            sudoku, safe_cells, order = restart_higher_order_put(sudoku, safe_cells, order, order_put)

            return solve(sudoku, safe_cells, order, order_put, go_back, rec + 1)
        
        row = row_col[0][0]
        col = row_col[1][0]
        
        num = sudoku[row][col]
        
        for n in range(num+1, 10):
            # if possible to put number in cell
            if if_possible(sudoku, row, col, n) == True:
                sudoku[row][col] = num
                safe_cells[row][col] = 1
                order[row][col] = order_put
                order_put = order_put = 1
            # if can't put different number in this cell
            elif if_possible(sudoku, row, col, n) == False and n == 9:
                # if it went back to the start there is an error
                if order_put == 1:
                    print("ERROR, can't go back. Actual order_put is {}".format(order_put))
                    return sudoku
                # if can't put in this cell go to previous order_put
                else:
                    go_back = True
                    order_put = order_put - 1

                    sudoku, safe_cells, order = restart_higher_order_put(sudoku, safe_cells, order, order_put)

                    return solve(sudoku, safe_cells, order, order_put, go_back, rec + 1)
                
    #if not going back
    else:
        count_table = np.zeros(81,dtype="int").reshape(9,9)

        for i in range(9):
            for j in range(9):
                if safe_cells[i][j] == 0:
                    count = 0
                    num = None

                    for n in range(1,10):
                        if if_possible(sudoku, i, j, n) == True:
                            count = count + 1
                            num = n

                    count_table[i][j] = count

                    # if there will be 1 possiblity it will be put in cell
                    if count == 1:
                        sudoku[i][j] = num
                        safe_cells[i][j] = 1
                        order[i][j] = order_put
                        # not incrementing order_put so all sure cells will be in the same step in history
                        go_back = False
        
        #error, there is no possibility to put in a number in any cell
        if count_table.sum() == 0:
            go_back = True
            order_put = order_put - 1

            sudoku, safe_cells, order = restart_higher_order_put(sudoku, safe_cells, order, order_put)

            return solve(sudoku, safe_cells, order, order_put, go_back, rec + 1)

        #if there won't be only 1 possibility in any cell do algorithm to put one of possibilities and go on and then go back at some point
        if 1 not in count_table and safe_cells.sum() != 81:    
            min_pos = count_table[count_table > 1].min()

            for i in range(9):
                for j in range(9):
                    if count_table[i][j] == min_pos:
                        for n in range(1,10):
                            if if_possible(sudoku, i, j, n) == True:
                                sudoku[i][j] = n
                                safe_cells[i][j] = 1
                                order[i][j] = order_put
                                order_put = order_put + 1
                                go_back = False
                                return solve(sudoku, safe_cells, order, order_put, go_back, rec + 1)
                                
            #print(sudoku)
            #print(safe_cells.sum())
            #print(min_pos)
    order_put = order_put + 1
            
    # ending if
    if safe_cells.sum() == 81:
        return sudoku
    else:
        return solve(sudoku, safe_cells, order, order_put, go_back, rec+1)

In [25]:
arr = np.array([[0,1,2,3],[6,3,3,9]])

In [27]:
res = np.where(arr==3)
print(len(res[0]))
print(res[0][0])
print(res[1][0])

3
0
3


In [133]:
#sudoku = clear_sudoku()
#sudoku = example_easy_sudoku()
sudoku = example_hard_sudoku() #works with threading
#sudoku = example_very_hard_sudoku() #doesn't work for now

In [134]:
sudoku

array([[8, 3, 2, 1, 0, 7, 0, 9, 6],
       [0, 0, 0, 0, 0, 0, 0, 5, 0],
       [7, 4, 5, 0, 9, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 7],
       [0, 0, 0, 8, 0, 1, 0, 0, 0],
       [0, 0, 4, 0, 0, 0, 5, 0, 1],
       [3, 0, 0, 0, 0, 5, 0, 6, 2],
       [0, 6, 0, 0, 0, 8, 0, 0, 4],
       [0, 0, 0, 7, 0, 3, 0, 8, 0]])

In [135]:
safe_cells = np.copy(sudoku)
np.place(safe_cells, safe_cells>0, [True])
print(safe_cells)
#print(sudoku)

[[1 1 1 1 0 1 0 1 1]
 [0 0 0 0 0 0 0 1 0]
 [1 1 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 1 0 1]
 [1 0 0 0 0 1 0 1 1]
 [0 1 0 0 0 1 0 0 1]
 [0 0 0 1 0 1 0 1 0]]


In [136]:
safe_cells.sum()

28

In [137]:
order = np.zeros(81,dtype="int").reshape(9,9)
order_put = 1
rec = 1

In [138]:
#%%timeit
solve(sudoku, safe_cells, order, order_put, False, rec)

Rec: 1
[[8 3 2 1 0 7 0 9 6]
 [0 0 0 0 0 0 0 5 0]
 [7 4 5 0 9 0 0 0 0]
 [0 0 0 0 0 0 0 0 7]
 [0 0 0 8 0 1 0 0 0]
 [0 0 4 0 0 0 5 0 1]
 [3 0 0 0 0 5 0 6 2]
 [0 6 0 0 0 8 0 0 4]
 [0 0 0 7 0 3 0 8 0]]
Safe_cells: [[1 1 1 1 0 1 0 1 1]
 [0 0 0 0 0 0 0 1 0]
 [1 1 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 1 0 1]
 [1 0 0 0 0 1 0 1 1]
 [0 1 0 0 0 1 0 0 1]
 [0 0 0 1 0 1 0 1 0]]
Order: [[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
Order_put: 1
Go_back: False
Rec: 2
[[8 3 2 1 0 7 4 9 6]
 [0 0 0 0 0 0 0 5 0]
 [7 4 5 0 9 0 0 0 0]
 [0 0 0 0 0 0 0 0 7]
 [0 0 0 8 0 1 0 0 0]
 [0 0 4 0 0 0 5 0 1]
 [3 0 0 0 0 5 0 6 2]
 [0 6 0 0 0 8 0 0 4]
 [0 0 0 7 0 3 0 8 0]]
Safe_cells: [[1 1 1 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 1 0]
 [1 1 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 1 0 1]
 [1 0 0 0 0 1 0 1 1]
 [0 1 0 0 0 1 0 0 1]

array([[8, 3, 2, 1, 0, 7, 4, 9, 6],
       [0, 0, 0, 0, 0, 0, 0, 5, 0],
       [7, 4, 5, 0, 9, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 7],
       [0, 0, 0, 8, 0, 1, 0, 0, 0],
       [0, 0, 4, 0, 0, 0, 5, 0, 1],
       [3, 0, 0, 0, 0, 5, 0, 6, 2],
       [0, 6, 0, 0, 0, 8, 0, 0, 4],
       [0, 0, 0, 7, 0, 3, 0, 8, 0]])

In [24]:
print(sud)

NameError: name 'sud' is not defined

In [15]:
# for easier sudoku
#%%time
#sudoku = solve(sudoku, on_start, 0, 0, 1, go_back=False, rec=0)

In [16]:
sudoku

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [17]:
threading.stack_size(67108864) # 64MB stack
sys.setrecursionlimit(2 ** 20) # something real big
                               # you actually hit the 64MB limit first
                               # going by other answers, could just use 2**32-1

In [18]:
# for harder sudoku
#%%time
# only new threads get the redefined stack size
thread = threading.Thread(target=solve, args = (sudoku, on_start, 0, 0, 1, False, 0, ))
thread.start()

Recursion: 0
Recursion: 1000
Recursion: 2000
Recursion: 3000
Recursion: 3437


In [19]:
sudoku

array([[1, 2, 3, 4, 5, 6, 7, 8, 9],
       [4, 5, 6, 7, 8, 9, 1, 2, 3],
       [7, 8, 9, 1, 2, 3, 4, 5, 6],
       [2, 1, 4, 3, 6, 5, 8, 9, 7],
       [3, 6, 5, 8, 9, 7, 2, 1, 4],
       [8, 9, 7, 2, 1, 4, 3, 6, 5],
       [5, 3, 1, 6, 4, 2, 9, 7, 8],
       [6, 4, 2, 9, 7, 8, 5, 3, 1],
       [9, 7, 8, 5, 3, 1, 6, 4, 2]])